# Classificação de texto com TensorFlow

In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow.keras import preprocessing

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
tf.__version__

### Obtendo e preparando os dados

In [ ]:
train_dir = "../dataset/v1/"

batch_size = 32
seed = 42
validation_split = 0.2

raw_train_ds = preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=validation_split,
    subset='training',
    seed=seed)

raw_val_ds = preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=validation_split,
    subset='validation',
    seed=seed)

In [ ]:
for index, label in enumerate(raw_train_ds.class_names):
  print(f"Label {index} corresponde a '{label}'")

In [ ]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(5):
    print("Texto: ", text_batch.numpy()[i][:50], '...')
    print("Label:", label_batch.numpy()[i])

### Tokenizer

In [ ]:
VOCAB_SIZE = 10000

vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='binary')

In [ ]:
train_text = raw_train_ds.map(lambda text, labels: text)

vectorize_layer.adapt(train_text)

In [ ]:
def binary_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [ ]:
train_ds = raw_train_ds.map(binary_vectorize_text)
val_ds = raw_val_ds.map(binary_vectorize_text)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
train_ds = configure_dataset(train_ds)
val_ds = configure_dataset(val_ds)

In [ ]:
model = tf.keras.Sequential([
                                    tf.keras.layers.Dense(5)
                                    ])

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

history = model.fit(
    train_ds, validation_data=val_ds, epochs=10)

In [ ]:
model.summary()

### Save the Model

In [ ]:
model_dir = "../app/model"

In [ ]:
export_model = tf.keras.Sequential(
    [vectorize_layer, model,
     tf.keras.layers.Activation('sigmoid')])

export_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=['accuracy'])

loss, accuracy = export_model.evaluate(raw_train_ds)
print("Accuracy: {:2.2%}".format(accuracy))

export_model.save(model_dir)

In [ ]:
loaded_model = tf.keras.models.load_model(model_dir)

In [ ]:
def get_string_labels(predicted_scores_batch):
  predicted_int_labels = tf.argmax(predicted_scores_batch, axis=1)
  predicted_labels = tf.gather(raw_train_ds.class_names, predicted_int_labels)
  return predicted_labels

In [ ]:
inputs = [
    """
    Congresso e STF criticam Bolsonaro por discurso
    """,
    """
    A CBF já começa a traçar o planejamento para as competições femininas no próximo ano. Um ponto entrou em debate em conversa com os clubes. A entidade deseja que todas as atletas que disputam a Série A1 do Brasileiro feminino tenham carteira assinada. A Confederação Brasileira de Futebol também planeja realizar torneios regionais, a partir de 2022 - ao estilo Rio/SP e Sul/Minas, entre outras regiões. A objetivo é ampliar um pouco mais o número de jogos no calendário, ainda carente ao longo do ano. A ideia de Copas não foi descartada para as temporadas seguintes.
    """,
    """
    A Câmara dos Deputados aprovou nesta quinta-feira (10) um projeto de lei que estabelece normas para combater e punir a violência política contra mulheres, inserindo novos dispositivos no Código Eleitoral e na Lei das Eleições. O texto ainda será analisado pelo Senado.

A proposta é uma reação aos casos de ataques a candidatas nas últimas eleições municipais.

O projeto classifica como violência política contra a mulher "toda ação, conduta ou omissão com a finalidade de impedir, obstaculizar, ou restringir os direitos políticos das mulheres".

O projeto determina que “as autoridades competentes privilegiarão o imediato exercício do direito violado, conferindo especial importância às declarações da vítima e aos elementos indiciários”.
    """
]
predicted_scores = loaded_model.predict(inputs)

predicted_labels = get_string_labels(predicted_scores)

print(predicted_labels)

for input, label in zip(inputs, predicted_labels):
  print("Text: ", input)
  print("Label: ", label.numpy())